In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [2]:
categories = ['EDUCATION', 'BUSINESS', 
       'DATING', 'SPORTS', 
       'WEATHER', 'FOOD_AND_DRINK', 
       'HEALTH_AND_FITNESS', 'BEAUTY', 
       'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 
       'SOCIAL', 'SHOPPING', 
       'PRODUCTIVITY', 'PHOTOGRAPHY', 
       'MEDICAL', 'PARENTING', 
       'EDUCATION', 'BEAUTY',
       'COMMUNICATION', 'TOOLS']

In [3]:
def get_links(data):
    '''Given html data this will extract all the links'''
    link_list = []
    for link in data.findAll('a'):
        get_link = link.get('href')
        link_list.append(get_link)
    return link_list

In [5]:
def webpage_html(url):
    r = requests.get(url)
    html = r.text
    return html

In [6]:
def google_scraper(categories):
    '''Will cycle through every category and then request the site, pass the soup to 
        get_links and then clean the list and append results to a new list in a dict
        format that has the format of {cat_name : [link list]}'''
    dicty = dict()
    for category in categories:
        url = f'https://play.google.com/store/apps/category/{category}/collection/topselling_free'
        html = webpage_html(url)
        soup = BeautifulSoup(html)
        links = get_links(soup)
        del links[0:107]
        clean = list(set(links))
        new_list = [app for app in clean if app.find('details') != -1]
        dicty[category] = new_list
        time.sleep(3)
    return dicty


In [7]:
cat = categories

In [8]:
stuff = google_scraper(cat)

In [9]:
stuff

{'EDUCATION': ['/store/apps/details?id=net.wordbit.enes',
  '/store/apps/details?id=com.smart.math.calculator',
  '/store/apps/details?id=org.khanacademy.android',
  '/store/apps/details?id=com.king.hindi.spanish.translator',
  '/store/apps/details?id=com.getmimo',
  '/store/apps/details?id=com.coco.ged',
  '/store/apps/details?id=com.duolingo',
  '/store/apps/details?id=com.babbel.mobile.android.es',
  '/store/apps/details?id=org.pbskids.video',
  '/store/apps/details?id=com.aofl.readingiq.google',
  '/store/apps/details?id=com.funpop.android_unicorncake',
  '/store/apps/details?id=com.quizlet.quizletandroid',
  '/store/apps/details?id=dollsforkids.coloringbook.fun',
  '/store/apps/details?id=net.appbeta.deepwebmobile',
  '/store/apps/details?id=com.yousician.yousician',
  '/store/apps/details?id=com.udemy.android',
  '/store/apps/details?id=curso.d.ingles.gratis',
  '/store/apps/details?id=com.instructure.candroid',
  '/store/apps/details?id=com.classdojo.android',
  '/store/apps/det

In [67]:
x = 0
for key, value in stuff.items():
    x+=len(value)
x


1080

In [35]:
def parse(listing_html, criteria):
    result = listing_html.find(class_=re.compile(r'.*%s' % criteria))
    if result:
        return result.text

In [61]:
def get_description(category_dict):
    dicty = dict()
    url = 'https://play.google.com'
    for category, link_list in category_dict.items():
        listy = list()
        for link in link_list:
            html = webpage_html(url+link)
            soup = BeautifulSoup(html)
            html1 = soup.find('div', {'class':"LXrl4c"})
            description = parse(html1, 'DWPxHb')
            time.sleep(1)
            listy.append(description)
        dicty[category] = listy
    return dicty              

In [62]:
all_desc = get_description(stuff)

In [2]:
import numpy as np

In [3]:
# # Save
# np.save('my_file.npy', all_desc) 

# # # Load
read_dictionary = np.load('my_file.npy').item()

In [68]:
x = 0
y = 0
for k,v in all_desc.items():
    x+= len(v)
    y += 1
y, x

(18, 1080)

In [18]:
edu  = read_dictionary

In [42]:
read_dictionary.keys()

18

In [14]:
import nltk
import sklearn

from nltk.collocations import *
from nltk import FreqDist, word_tokenize
import string, re
from nltk.stem.snowball import SnowballStemmer

In [15]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"

In [16]:
# stop words
from nltk.corpus import stopwords
stopwords.words("english")

stop_words = set(stopwords.words('english'))

In [17]:
# stem words
stemmer = SnowballStemmer("english")

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [19]:
review_list = []
for c, d in edu.items():
    for review in d:
        art_tokens_raw = nltk.regexp_tokenize(review, pattern)
        art_tokens = [i.lower() for i in art_tokens_raw]
        art_tokens_stopped = [w for w in art_tokens if not w in stop_words]
        art_stemmed = [stemmer.stem(word) for word in art_tokens_stopped]
        cleaned = ' '.join(art_stemmed)
        review_list.append(cleaned)

In [20]:
response = tfidf.fit_transform(review_list )

df = pd.DataFrame(response.toarray(), columns=tfidf.get_feature_names())


In [23]:
cat = ['EDUCATION', 'BUSINESS', 
       'DATING', 'SPORTS', 
       'WEATHER', 'FOOD_AND_DRINK', 
       'HEALTH_AND_FITNESS', 'BEAUTY', 
       'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 
       'SOCIAL', 'SHOPPING', 
       'PRODUCTIVITY', 'PHOTOGRAPHY', 
       'MEDICAL', 'PARENTING', 
       'EDUCATION', 'BEAUTY',
       'COMMUNICATION', 'TOOLS']

In [43]:
x = read_dictionary.keys()
new_list = []
for c in x: 
    s = [c] * 60
    new_list += s

In [44]:
new_list

['EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'EDUCATION',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BUSINESS',
 'BU

In [39]:
df.shape

(1080, 13205)

In [45]:
len(new_list)

1080

In [46]:
df['labels'] = new_list

In [50]:
df.tail()

,aa,aac,aaptiv,aarp,aask,ab,abajo,abandon,abc,abcmous,...,zoomterrain,zoomwithushav,zoosk,zte,zulili,zulu,zulufor,zulupermiss,zumba,labels
1075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
1079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOOLS
